# Crear una aplicación web de análisis de opiniones

## Usando PyTorch y SageMaker

Programa de aprendizaje profundo Nanodegree | Despliegue

Ahora que tenemos una comprensión básica de cómo funciona SageMaker, intentaremos usarlo para construir un proyecto completo de principio a fin. Nuestro objetivo será tener una página web simple que un usuario pueda usar para ingresar una reseña de la película. La página web enviará la revisión a nuestro modelo implementado, que predecirá el sentimiento de la revisión ingresada.

## Instrucciones
Ya se le ha proporcionado un código de plantilla, y deberá implementar una funcionalidad adicional para completar con éxito este cuaderno. No necesitará modificar el código incluido más allá de lo solicitado. Las secciones que comienzan con 'TODO' en el encabezado indican que debe completar o implementar alguna parte dentro de ellas. Se proporcionarán instrucciones para cada sección y los detalles de la implementación están marcados en el bloque de código con un # TODO: ... comentario. ¡Asegúrese de leer las instrucciones cuidadosamente!

Además de implementar el código, habrá preguntas para responder que se relacionan con la tarea y su implementación. Cada sección donde responderá una pregunta está precedida por un encabezado 'Pregunta:'. Lea atentamente cada pregunta y proporcione su respuesta debajo del encabezado 'Respuesta:' editando la celda Markdown.

**Nota**: Las celdas de código y Markdown se pueden ejecutar usando el atajo de teclado Shift + Enter. Además, una celda se puede editar haciendo clic en ella (haciendo doble clic en las celdas Markdown) o presionando Entrar mientras está resaltada.

## Bosquejo general
Recuerde el esquema general de los proyectos de SageMaker utilizando una instancia de cuaderno.

1. Descargue o recupere los datos.
2. Procesar / preparar los datos.
3. Suba los datos procesados a S3.
4. Entrenar a un modelo elegido.
5. Pruebe el modelo entrenado (generalmente usando un trabajo de transformación por lotes).
6. Implemente el modelo entrenado.
7. Use el modelo desplegado.
Para este proyecto, seguirá los pasos del esquema general con algunas modificaciones.

Primero, no probará el modelo en su propio paso. Todavía estará probando el modelo, sin embargo, lo hará implementando su modelo y luego utilizando el modelo implementado enviándole los datos de prueba. Una de las razones para hacerlo es para que pueda asegurarse de que su modelo implementado funcione correctamente antes de seguir adelante.

Además, implementará y utilizará su modelo entrenado por segunda vez. En la segunda iteración, personalizará la forma en que se implementa su modelo entrenado al incluir parte de su propio código. Además, su modelo recién implementado se utilizará en la aplicación web de análisis de sentimientos.

## Paso 1: descargando los datos
Al igual que en el cuaderno XGBoost en SageMaker, utilizaremos el conjunto de datos de IMDb

In [ ]:
%mkdir ../data
!wget -O ../data/aclImdb_v1.tar.gz http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -zxf ../data/aclImdb_v1.tar.gz -C ../data

## Paso 2: preparación y procesamiento de los datos
Además, como en el cuaderno XGBoost, haremos un procesamiento inicial de datos. Los primeros pasos son los mismos que en el ejemplo XGBoost. Para comenzar, leeremos cada una de las revisiones y las combinaremos en una sola estructura de entrada. Luego, dividiremos el conjunto de datos en un conjunto de entrenamiento y un conjunto de prueba.

In [ ]:
import os
import glob

def read_imdb_data(data_dir='../data/aclImdb'):
    data = {}
    labels = {}
    
    for data_type in ['train', 'test']:
        data[data_type] = {}
        labels[data_type] = {}
        
        for sentiment in ['pos', 'neg']:
            data[data_type][sentiment] = []
            labels[data_type][sentiment] = []
            
            path = os.path.join(data_dir, data_type, sentiment, '*.txt')
            files = glob.glob(path)
            
            for f in files:
                with open(f) as review:
                    data[data_type][sentiment].append(review.read())
                    # Here we represent a positive review by '1' and a negative review by '0'
                    labels[data_type][sentiment].append(1 if sentiment == 'pos' else 0)
                    
            assert len(data[data_type][sentiment]) == len(labels[data_type][sentiment]), \
                    "{}/{} data size does not match labels size".format(data_type, sentiment)
                
    return data, labels

In [ ]:
data, labels = read_imdb_data()
print("IMDB reviews: train = {} pos / {} neg, test = {} pos / {} neg".format(
            len(data['train']['pos']), len(data['train']['neg']),
            len(data['test']['pos']), len(data['test']['neg'])))

Ahora que hemos leído los datos de entrenamiento y pruebas sin procesar del conjunto de datos descargado, combinaremos las revisiones positivas y negativas y mezclaremos los registros resultantes.

In [ ]:
from sklearn.utils import shuffle

def prepare_imdb_data(data, labels):
    """Prepare training and test sets from IMDb movie reviews."""
    
    #Combine positive and negative reviews and labels
    data_train = data['train']['pos'] + data['train']['neg']
    data_test = data['test']['pos'] + data['test']['neg']
    labels_train = labels['train']['pos'] + labels['train']['neg']
    labels_test = labels['test']['pos'] + labels['test']['neg']
    
    #Shuffle reviews and corresponding labels within training and test sets
    data_train, labels_train = shuffle(data_train, labels_train)
    data_test, labels_test = shuffle(data_test, labels_test)
    
    # Return a unified training data, test data, training labels, test labets
    return data_train, data_test, labels_train, labels_test

In [ ]:
train_X, test_X, train_y, test_y = prepare_imdb_data(data, labels)
print("IMDb reviews (combined): train = {}, test = {}".format(len(train_X), len(test_X)))

Ahora que tenemos nuestros conjuntos de entrenamiento y prueba unificados y preparados, deberíamos hacer una verificación rápida y ver un ejemplo de los datos en los que se entrenará nuestro modelo. En general, es una buena idea, ya que le permite ver cómo cada uno de los pasos de procesamiento adicionales afecta las revisiones y también garantiza que los datos se hayan cargado correctamente.

In [ ]:
print(train_X[100])
print(train_y[100])

El primer paso para procesar las revisiones es asegurarse de que se eliminen las etiquetas html que aparezcan. Además, deseamos tokenizar nuestro aporte, de esa manera palabras como entretenido y entretenido se consideran lo mismo con respecto al análisis de sentimientos.

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import *

import re
from bs4 import BeautifulSoup

def review_to_words(review):
    nltk.download("stopwords", quiet=True)
    stemmer = PorterStemmer()
    
    text = BeautifulSoup(review, "html.parser").get_text() # Remove HTML tags
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower()) # Convert to lower case
    words = text.split() # Split string into words
    words = [w for w in words if w not in stopwords.words("english")] # Remove stopwords
    words = [PorterStemmer().stem(w) for w in words] # stem
    
    return words

El método review_to_words definido anteriormente utiliza BeautifulSoup para eliminar las etiquetas html que aparecen y utiliza el paquete nltk para simular las revisiones. Para verificar que sabemos cómo funciona todo, intente aplicar review_to_words a una de las revisiones del conjunto de capacitación.

In [ ]:
# TODO: aplicar review_to_words a una revisión (train_X[100] o cualquier otra revisión)
review_to_words(train_X[100])

**Pregunta**: Anteriormente mencionamos que el método review_to_words elimina el formato html y nos permite tokenizar las palabras encontradas en una revisión, por ejemplo, convertir entretenidos y entretenidos en entretener para que sean tratados como si fueran la misma palabra. ¿Qué más, si hay algo, le hace este método a la entrada?

**Respuesta**:

El siguiente método aplica el método review_to_words a cada una de las revisiones en los conjuntos de datos de capacitación y prueba. Además, almacena en caché los resultados. Esto se debe a que realizar este paso de procesamiento puede llevar mucho tiempo. De esta manera, si no puede completar el cuaderno en la sesión actual, puede regresar sin necesidad de procesar los datos por segunda vez.

In [ ]:
import pickle

cache_dir = os.path.join("../cache", "sentiment_analysis")  # where to store cache files
os.makedirs(cache_dir, exist_ok=True)  # ensure cache directory exists

def preprocess_data(data_train, data_test, labels_train, labels_test,
                    cache_dir=cache_dir, cache_file="preprocessed_data.pkl"):
    """Convert each review to words; read from cache if available."""

    # If cache_file is not None, try to read from it first
    cache_data = None
    if cache_file is not None:
        try:
            with open(os.path.join(cache_dir, cache_file), "rb") as f:
                cache_data = pickle.load(f)
            print("Read preprocessed data from cache file:", cache_file)
        except:
            pass  # unable to read from cache, but that's okay
    
    # If cache is missing, then do the heavy lifting
    if cache_data is None:
        # Preprocess training and test data to obtain words for each review
        #words_train = list(map(review_to_words, data_train))
        #words_test = list(map(review_to_words, data_test))
        words_train = [review_to_words(review) for review in data_train]
        words_test = [review_to_words(review) for review in data_test]
        
        # Write to cache file for future runs
        if cache_file is not None:
            cache_data = dict(words_train=words_train, words_test=words_test,
                              labels_train=labels_train, labels_test=labels_test)
            with open(os.path.join(cache_dir, cache_file), "wb") as f:
                pickle.dump(cache_data, f)
            print("Wrote preprocessed data to cache file:", cache_file)
    else:
        # Unpack data loaded from cache file
        words_train, words_test, labels_train, labels_test = (cache_data['words_train'],
                cache_data['words_test'], cache_data['labels_train'], cache_data['labels_test'])
    
    return words_train, words_test, labels_train, labels_test

In [ ]:
# Preprocess data
train_X, test_X, train_y, test_y = preprocess_data(train_X, test_X, train_y, test_y)

## Transforma los datos
En el cuaderno XGBoost transformamos los datos de su representación de palabras a una representación de características de bolsa de palabras. Para el modelo que vamos a construir en este cuaderno, construiremos una representación de características que es muy similar. Para comenzar, representaremos cada palabra como un número entero. Por supuesto, algunas de las palabras que aparecen en las revisiones ocurren con poca frecuencia y, por lo tanto, es probable que no contengan mucha información para el análisis de sentimientos. La forma en que trataremos este problema es que arreglaremos el tamaño de nuestro vocabulario de trabajo y solo incluiremos las palabras que aparecen con más frecuencia. Luego combinaremos todas las palabras poco frecuentes en una sola categoría y, en nuestro caso, la etiquetaremos como 1.

Como utilizaremos una red neuronal recurrente, será conveniente que la duración de cada revisión sea la misma. Para hacer esto, fijaremos un tamaño para nuestras revisiones y luego rellenaremos revisiones cortas con la categoría 'sin palabra' (que etiquetaremos como 0) y truncaremos las revisiones largas.

### (TODO) Crear un diccionario de palabras
Para comenzar, necesitamos construir una forma de mapear las palabras que aparecen en las revisiones a enteros. Aquí fijamos el tamaño de nuestro vocabulario (incluidas las categorías 'sin palabra' e 'infrecuente') en 5000, pero es posible que desee cambiar esto para ver cómo afecta al modelo.

> **TODO**: Complete la implementación para el método build_dict () a continuación. Tenga en cuenta que aunque vocab_size esté configurado en 5000, solo queremos construir un mapeo para las 4998 palabras que aparecen con más frecuencia. Esto se debe a que queremos reservar las etiquetas especiales 0 para 'sin palabra' y 1 para 'palabra infrecuente'.

In [ ]:
import numpy as np

def build_dict(data, vocab_size = 5000):
    """Construye y devuelve un diccionario que asigna cada una
    de las palabras que aparecen con más frecuencia a un número entero único"""
    
     #TODO: Determine con qué frecuencia aparece cada palabra en `data`.
        #Tenga en cuenta que `data` es una lista de oraciones y que una oración es una lista de palabras.
    word_count = {}
    # Un diccionario que almacena las palabras que aparecen en las reseñas junto con la frecuencia con la que aparecen
    for review in data:
        if word in review:
            word_count[word] += 1
        else:
            word_count[word] = 1
     # TODO: ordena las palabras encontradas en `data` para que sorted_words [0] sea la palabra que aparece con más frecuencia y
     # sorted_words [-1] es la palabra que aparece con menos frecuencia.
    sorted_words = None
    sorted_words = sorted(word_count, key=word_count.get, reverse=True)
    
    word_dict = {} # Esto es lo que estamos construyendo, un diccionario que traduce palabras en enteros
    
    for idx, word in enumerate(sorted_words[:vocab_size - 2]):
        word_dict[word] = idx + 2
    # El -2 es para que ahorremos espacio para las etiquetas 'no word' 'infrequent'
    return word_dict
    

In [ ]:
word_dict = build_dict(train_X)

**Pregunta**: ¿Cuáles son las cinco palabras que aparecen con más frecuencia (tokenizadas) en el conjunto de entrenamiento? ¿Tiene sentido que estas palabras aparezcan con frecuencia en el conjunto de entrenamiento?

**Respuesta**:

In [ ]:
# TODO: use este espacio para determinar las cinco palabras que aparecen
# con más frecuencia en el conjunto de entrenamiento.


## Guarde word_dict
Más adelante, cuando construyamos un punto final que procese una revisión enviada, necesitaremos usar el word_dict que hemos creado. Como tal, lo guardaremos en un archivo ahora para uso futuro.

In [ ]:
data_dir = '../data/pytorch' # Carpeta para guardar el archivo
if not os.path.exists(data_dir): # Comprobar si la carpeta existe
    os.makedirs(data_dir)

In [ ]:
with open(os.path.join(data_dir, 'word_dict.pkl'), "wb") as f:
    pickle.dump(word_dict, f)

## Transforma las reseñas
Ahora que tenemos nuestro diccionario de palabras que nos permite transformar las palabras que aparecen en las revisiones en números enteros, es hora de usarlo y convertir nuestras revisiones a su representación de secuencia entera, asegurándose de rellenar o truncar a una longitud fija, que en nuestro caso es 500.

In [ ]:
def convert_and_pad(word_dict, sentence, pad=500):
    NOWORD = 0 # We will use 0 to represent the 'no word' category
    INFREQ = 1 # and we use 1 to represent the infrequent words, i.e., words not appearing in word_dict
    
    working_sentence = [NOWORD] * pad
    
    for word_index, word in enumerate(sentence[:pad]):
        if word in word_dict:
            working_sentence[word_index] = word_dict[word]
        else:
            working_sentence[word_index] = INFREQ
            
    return working_sentence, min(len(sentence), pad)

def convert_and_pad_data(word_dict, data, pad=500):
    result = []
    lengths = []
    
    for sentence in data:
        converted, leng = convert_and_pad(word_dict, sentence, pad)
        result.append(converted)
        lengths.append(leng)
        
    return np.array(result), np.array(lengths)

In [ ]:
train_X, train_X_len = convert_and_pad_data(word_dict, train_X)
test_X, test_X_len = convert_and_pad_data(word_dict, test_X)

Como una verificación rápida para asegurarse de que las cosas funcionan como se esperaba, verifique cómo se ve una de las revisiones en el conjunto de capacitación después de haber sido procesada. ¿Esto parece razonable? ¿Cuál es la duración de una revisión en el conjunto de capacitación?

In [ ]:
# Use esta celda para examinar una de las revisiones procesadas para asegurarse de que todo funcione según lo previsto.

**Pregunta**: En las celdas de arriba usamos los métodos preprocess_data y convert_and_pad_data para procesar tanto el conjunto de entrenamiento como el de pruebas. ¿Por qué o por qué no podría ser un problema?

**Respuesta**:

## Paso 3: sube los datos a S3
Al igual que en el cuaderno XGBoost, necesitaremos cargar el conjunto de datos de entrenamiento en S3 para que nuestro código de entrenamiento pueda acceder a él. Por ahora lo guardaremos localmente y lo subiremos a S3 más adelante.

### Guarde el conjunto de datos de entrenamiento procesado localmente
Es importante tener en cuenta el formato de los datos que estamos guardando, ya que necesitaremos saberlo cuando escribamos el código de capacitación. En nuestro caso, cada fila del conjunto de datos tiene la etiqueta del formulario, longitud, revisión [500] donde revisión [500] es una secuencia de 500 enteros que representan las palabras en la revisión.

In [ ]:
import pandas as pd
    
pd.concat([pd.DataFrame(train_y), pd.DataFrame(train_X_len), pd.DataFrame(train_X)], axis=1) \
        .to_csv(os.path.join(data_dir, 'train.csv'), header=False, index=False)

## Subiendo los datos de entrenamiento
A continuación, debemos cargar los datos de entrenamiento al depósito S3 predeterminado de SageMaker para que podamos proporcionar acceso a ellos mientras entrenamos nuestro modelo.

In [ ]:
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/sentiment_rnn'

role = sagemaker.get_execution_role()

In [ ]:
input_data = sagemaker_session.upload_data(path=data_dir, bucket=bucket, key_prefix=prefix)

**NOTA**: La celda de arriba carga todo el contenido de nuestro directorio de datos. Esto incluye el archivo word_dict.pkl. Esto es afortunado ya que lo necesitaremos más adelante cuando creamos un punto final que acepte una revisión arbitraria. Por ahora, solo tomaremos nota del hecho de que reside en el directorio de datos (y también en el grupo de entrenamiento S3) y que necesitaremos asegurarnos de que se guarde en el directorio del modelo.

## Paso 4: Construye y entrena el modelo PyTorch
En el cuaderno XGBoost discutimos qué es un modelo en el marco de SageMaker. En particular, un modelo comprende tres objetos.

* Artefactos modelo,
* Código de entrenamiento, y
* Código de inferencia,
cada uno de los cuales interactúa entre sí. En el ejemplo de XGBoost, utilizamos el código de capacitación e inferencia proporcionado por Amazon. Aquí todavía usaremos contenedores proporcionados por Amazon con el beneficio adicional de poder incluir nuestro propio código personalizado.

Comenzaremos implementando nuestra propia red neuronal en PyTorch junto con un script de entrenamiento. Para los propósitos de este proyecto, hemos proporcionado el objeto modelo necesario en el archivo model.py, dentro de la carpeta del tren. Puede ver la implementación proporcionada ejecutando la celda a continuación.

In [ ]:
!pygmentize train/model.py

La conclusión importante de la implementación proporcionada es que hay tres parámetros que podemos desear ajustar para mejorar el rendimiento de nuestro modelo. Estas son la dimensión de incrustación, la dimensión oculta y el tamaño del vocabulario. Es probable que queramos que estos parámetros sean configurables en el script de entrenamiento para que, si deseamos modificarlos, no necesitemos modificar el script en sí. Veremos cómo hacer esto más adelante. Para comenzar, escribiremos algunos de los códigos de capacitación en el cuaderno para que podamos diagnosticar más fácilmente cualquier problema que surja.

Primero, cargaremos una pequeña porción del conjunto de datos de entrenamiento para usar como muestra. Trataría mucho tiempo intentar entrenar el modelo por completo en el portátil, ya que no tenemos acceso a una gpu y la instancia de cálculo que estamos utilizando no es particularmente poderosa. Sin embargo, podemos trabajar en una pequeña parte de los datos para tener una idea de cómo se comporta nuestro script de entrenamiento.

In [ ]:
import torch
import torch.utils.data

# Read in only the first 250 rows
train_sample = pd.read_csv(os.path.join(data_dir, 'train.csv'), header=None, names=None, nrows=250)

# Turn the input pandas dataframe into tensors
train_sample_y = torch.from_numpy(train_sample[[0]].values).float().squeeze()
train_sample_X = torch.from_numpy(train_sample.drop([0], axis=1).values).long()

# Build the dataset
train_sample_ds = torch.utils.data.TensorDataset(train_sample_X, train_sample_y)
# Build the dataloader
train_sample_dl = torch.utils.data.DataLoader(train_sample_ds, batch_size=50)

## (TODO) Escribiendo el método de entrenamiento
Luego necesitamos escribir el código de entrenamiento en sí. Esto debería ser muy similar a los métodos de entrenamiento que ha escrito antes para entrenar modelos PyTorch. Dejaremos algunos aspectos difíciles como guardar / cargar modelos y cargar parámetros hasta un poco más tarde.

In [ ]:
def train(model, train_loader, epochs, optimizer, loss_fn, device):
    for epoch in range(1, epochs + 1):
        model.train()
        total_loss = 0
        for batch in train_loader:         
            batch_X, batch_y = batch
            
            batch_X = batch_X.to(device)
            batch_y = batch_y.to(device)
            
            # TODO: Complete this train method to train the model provided.
            
            total_loss += loss.data.item()
        print("Epoch: {}, BCELoss: {}".format(epoch, total_loss / len(train_loader)))

Suponiendo que tenemos el método de entrenamiento anterior, probaremos que está funcionando escribiendo un poco de código en el cuaderno que ejecuta nuestro método de entrenamiento en el pequeño conjunto de entrenamiento de muestra que cargamos anteriormente. La razón para hacer esto en el cuaderno es para que tengamos la oportunidad de corregir cualquier error que surja temprano cuando sea más fácil de diagnosticar.

In [ ]:
import torch.optim as optim
from train.model import LSTMClassifier

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LSTMClassifier(32, 100, 5000).to(device)
optimizer = optim.Adam(model.parameters())
loss_fn = torch.nn.BCELoss()

train(model, train_sample_dl, 5, optimizer, loss_fn, device)

Para construir un modelo PyTorch usando SageMaker, debemos proporcionar a SageMaker un script de entrenamiento. Opcionalmente, podemos incluir un directorio que se copiará en el contenedor y desde el cual se ejecutará nuestro código de capacitación. Cuando se ejecute el contenedor de entrenamiento, verificará el directorio cargado (si hay uno) para ver un archivo require.txt e instalará las bibliotecas Python necesarias, luego de lo cual se ejecutará el script de entrenamiento.

### (TODO) Entrenando al modelo
Cuando se construye un modelo PyTorch en SageMaker, se debe especificar un punto de entrada. Este es el archivo Python que se ejecutará cuando se entrene el modelo. Dentro del directorio de trenes hay un archivo llamado train.py que se ha proporcionado y que contiene la mayor parte del código necesario para entrenar nuestro modelo. Lo único que falta es la implementación del método train () que escribió anteriormente en este cuaderno.

**TODO**: copie el método train() escrito anteriormente y péguelo en el archivo train/train.py cuando sea necesario.

La forma en que SageMaker pasa los hiperparámetros al script de entrenamiento es a través de argumentos. Estos argumentos se pueden analizar y utilizar en el script de entrenamiento. Para ver cómo se hace esto, eche un vistazo al archivo train / train.py proporcionado.

In [ ]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point="train.py",
                    source_dir="train",
                    role=role,
                    framework_version='0.4.0',
                    train_instance_count=1,
                    train_instance_type='ml.p2.xlarge',
                    hyperparameters={
                        'epochs': 10,
                        'hidden_dim': 200,
                    })

In [ ]:
estimator.fit({'training': input_data})

### Paso 5: Prueba del modelo
Como se menciona en la parte superior de este cuaderno, probaremos este modelo al implementarlo primero y luego enviar los datos de prueba al punto final implementado. Haremos esto para asegurarnos de que el modelo implementado funcione correctamente.

### Paso 6: Implemente el modelo para probar
Ahora que hemos entrenado nuestro modelo, nos gustaría probarlo para ver cómo funciona. Actualmente, nuestro modelo toma la entrada de la forma review_length, review[500] donde review[500] es una secuencia de 500 enteros que describen las palabras presentes en la revisión, codificadas usando word_dict. Afortunadamente para nosotros, SageMaker proporciona un código de inferencia incorporado para modelos con entradas simples como esta.

Sin embargo, hay una cosa que debemos proporcionar, y es una función que carga el modelo guardado. Esta función debe llamarse model_fn() y toma como único parámetro una ruta al directorio donde se almacenan los artefactos del modelo. Esta función también debe estar presente en el archivo python que especificamos como punto de entrada. En nuestro caso, se ha proporcionado la función de carga del modelo y, por lo tanto, no es necesario realizar cambios.

NOTA: Cuando se ejecuta el código de inferencia incorporado, debe importar el método model_fn() del archivo train.py. Es por eso que el código de entrenamiento está envuelto en un protector principal (es decir, si __name__ == '__main__':)

Dado que no necesitamos cambiar nada en el código que se cargó durante el entrenamiento, simplemente podemos implementar el modelo actual tal como está.

**NOTA**: Al implementar un modelo, le está pidiendo a SageMaker que inicie una instancia de proceso que esperará a que se le envíen los datos. Como resultado, esta instancia de proceso continuará ejecutándose hasta que la cierre. Es importante saber esto, ya que el costo de un punto final implementado depende de cuánto tiempo ha estado funcionando.

En otras palabras **¡Si ya no estás usando un punto final desplegado, apágalo!**

**TODO**: Implemente el modelo entrenado.

In [ ]:
# TODO: Implemente el modelo entrenado

### Paso 7 - Use el modelo para probar
Una vez implementados, podemos leer los datos de prueba y enviarlos a nuestro modelo implementado para obtener algunos resultados. Una vez que recopilamos todos los resultados, podemos determinar qué tan preciso es nuestro modelo.

In [ ]:
test_X = pd.concat([pd.DataFrame(test_X_len), pd.DataFrame(test_X)], axis=1)

In [ ]:
# Dividimos los datos en fragmentos y enviamos cada fragmento por separado, acumulando los resultados.

def predict(data, rows=512):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = np.array([])
    for array in split_array:
        predictions = np.append(predictions, predictor.predict(array))
    
    return predictions

In [ ]:
predictions = predict(test_X.values)
predictions = [round(num) for num in predictions]

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(test_y, predictions)

**Pregunta**: ¿Cómo se compara este modelo con el modelo XGBoost que creó anteriormente? ¿Por qué podrían estos dos modelos funcionar de manera diferente en este conjunto de datos? ¿Cuál crees que es mejor para el análisis de sentimientos?

**Respuesta**:

### (TODO) Más pruebas
Ahora tenemos un modelo entrenado que se ha implementado y al que podemos enviar revisiones procesadas y que devuelve el sentimiento predicho. Sin embargo, en última instancia, nos gustaría poder enviar a nuestro modelo una revisión sin procesar. Es decir, nos gustaría enviar la revisión en sí como una cadena. Por ejemplo, supongamos que deseamos enviar la siguiente revisión a nuestro modelo.

In [ ]:
test_review = 'The simplest pleasures in life are the best, and this film is one of them. Combining a rather basic storyline of love and adventure this movie transcends the usual weekend fair with wit and unmitigated charm.'

La pregunta que ahora necesitamos responder es, ¿cómo enviamos esta revisión a nuestro modelo?

Recuerde en la primera sección de este cuaderno que hicimos un montón de procesamiento de datos para el conjunto de datos de IMDb. En particular, hicimos dos cosas específicas a las revisiones proporcionadas.

Se eliminaron las etiquetas html y se obtuvo la entrada.
Codificó la revisión como una secuencia de enteros usando word_dict
Para procesar la revisión, necesitaremos repetir estos dos pasos.

**TODO**: Usando los métodos review_to_words y convert_and_pad de la sección uno, convierta test_review en una matriz numpy test_data adecuada para enviar a nuestro modelo. Recuerde que nuestro modelo espera la entrada del formulario review_length, review[500].

In [ ]:
# TODO: Convierta test_review en un formulario que pueda usar el modelo y guarde los resultados en test_data
test_data = None

Ahora que hemos procesado la revisión, podemos enviar la matriz resultante a nuestro modelo para predecir el sentimiento de la revisión.

In [ ]:
predictor.predict(test_data)

Dado que el valor de retorno de nuestro modelo es cercano a 1, podemos estar seguros de que la revisión que enviamos es positiva.

### Eliminar el punto final
Por supuesto, al igual que en el portátil XGBoost, una vez que hemos implementado un punto final, continúa ejecutándose hasta que le decimos que se cierre. Como hemos terminado de usar nuestro punto final por ahora, podemos eliminarlo.

In [ ]:
estimator.delete_endpoint()

### Paso 6 (nuevamente) - Implemente el modelo para la aplicación web
Ahora que sabemos que nuestro modelo está funcionando, es hora de crear un código de inferencia personalizado para que podamos enviar al modelo una revisión que no se haya procesado y que determine el sentimiento de la revisión.

Como vimos anteriormente, de manera predeterminada, el estimador que creamos, cuando se implementa, usará el script de entrada y el directorio que proporcionamos al crear el modelo. Sin embargo, dado que ahora deseamos aceptar una cadena como entrada y nuestro modelo espera una revisión procesada, necesitamos escribir un código de inferencia personalizado.

Almacenaremos el código que escribimos en el directorio de servicio. En este directorio se proporciona el archivo model.py que utilizamos para construir nuestro modelo, un archivo utils.py que contiene las funciones de preprocesamiento review_to_words y convert_and_pad que utilizamos durante el procesamiento de datos inicial, y predict.py, el archivo que contendrá nuestro código de inferencia personalizado. Tenga en cuenta también que require.txt está presente, lo que le dirá a SageMaker qué bibliotecas de Python requiere nuestro código de inferencia personalizado.

Al implementar un modelo PyTorch en SageMaker, se espera que proporcione cuatro funciones que utilizará el contenedor de inferencia de SageMaker.

* model_fn: esta función es la misma que utilizamos en el script de entrenamiento y le dice a SageMaker cómo cargar nuestro modelo.
* input_fn: esta función recibe la entrada serializada sin formato que se ha enviado al punto final del modelo y su trabajo es deserializar y hacer que la entrada esté disponible para el código de inferencia.
* output_fn: esta función toma la salida del código de inferencia y su trabajo es serializar esta salida y devolverla al llamante del punto final del modelo.
* predic_fn: el corazón del script de inferencia, aquí es donde se realiza la predicción real y es la función que deberá completar.

Para el sitio web simple que estamos construyendo durante este proyecto, los métodos input_fn y output_fn son relativamente sencillos. Solo requerimos poder aceptar una cadena como entrada y esperamos devolver un solo valor como salida. Sin embargo, puede imaginar que en una aplicación más compleja la entrada o la salida pueden ser datos de imagen u otros datos binarios que requerirían un cierto esfuerzo para serializar.

### (TODO) Escribir código de inferencia
Antes de escribir nuestro código de inferencia personalizado, comenzaremos por echar un vistazo al código que se ha proporcionado.

In [ ]:
!pygmentize serve/predict.py

Como se mencionó anteriormente, el método model_fn es el mismo que el provisto en el código de entrenamiento y los métodos input_fn y output_fn son muy simples y su tarea será completar el método predict_fn. Asegúrese de guardar el archivo completado como predict.py en el directorio de servicio.

**TODO**: Complete el método predict_fn() en el archivo serve/predict.py.

### Implementando el modelo
Ahora que se ha escrito el código de inferencia personalizado, crearemos e implementaremos nuestro modelo. Para comenzar, necesitamos construir un nuevo objeto PyTorchModel que apunte a los artefactos del modelo creados durante el entrenamiento y también al código de inferencia que deseamos usar. Entonces podemos llamar al método de implementación para iniciar el contenedor de implementación.

**NOTA**: El comportamiento predeterminado para un modelo PyTorch implementado es asumir que cualquier entrada que se pase al predictor es una matriz numpy. En nuestro caso, queremos enviar una cadena, por lo que necesitamos construir un contenedor simple alrededor de la clase RealTimePredictor para acomodar cadenas simples. En una situación más complicada, es posible que desee proporcionar un objeto de serialización, por ejemplo, si desea enviar datos de imagen.

In [ ]:
from sagemaker.predictor import RealTimePredictor
from sagemaker.pytorch import PyTorchModel

class StringPredictor(RealTimePredictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super(StringPredictor, self).__init__(endpoint_name, sagemaker_session, content_type='text/plain')

model = PyTorchModel(model_data=estimator.model_data,
                     role = role,
                     framework_version='0.4.0',
                     entry_point='predict.py',
                     source_dir='serve',
                     predictor_cls=StringPredictor)
predictor = model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

### Prueba del modelo
Ahora que hemos implementado nuestro modelo con el código de inferencia personalizado, debemos probar para ver si todo funciona. Aquí probamos nuestro modelo cargando las primeras 250 críticas positivas y negativas y las enviamos al punto final, luego recopilamos los resultados. La razón para enviar solo algunos de los datos es que la cantidad de tiempo que le toma a nuestro modelo procesar la entrada y luego realizar la inferencia es bastante larga, por lo que probar todo el conjunto de datos sería prohibitivo.

In [ ]:
import glob

def test_reviews(data_dir='../data/aclImdb', stop=250):
    
    results = []
    ground = []
    
    # Nos aseguramos de evaluar las críticas positivas y negativas.  
    for sentiment in ['pos', 'neg']:
        
        path = os.path.join(data_dir, 'test', sentiment, '*.txt')
        files = glob.glob(path)
        
        files_read = 0
        
        print('Starting ', sentiment, ' files')
        
        # Iterar a través de los archivos y enviarlos al predictor
        for f in files:
            with open(f) as review:
                # Primero, almacenamos la verdad básica (fue la revisión positiva o negativa)
                if sentiment == 'pos':
                    ground.append(1)
                else:
                    ground.append(0)
                # Lea en la revisión y convierta a 'utf-8' para la transmisión a través de HTTP
                review_input = review.read().encode('utf-8')
                # Envíe la revisión al predictor y almacene los resultados.
                results.append(int(predictor.predict(review_input)))
                
            # Enviar reseñas a nuestro punto final de una en una lleva un tiempo,
            # por lo que solo enviamos una pequeña cantidad de reseñas
            files_read += 1
            if files_read == stop:
                break
            
    return ground, results

In [ ]:
ground, results = test_reviews()

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(ground, results)

Como prueba adicional, podemos intentar enviar el test_review que vimos anteriormente.

In [ ]:
predictor.predict(test_review)

Ahora que sabemos que nuestro punto final funciona como se esperaba, podemos configurar la página web que interactuará con él. Si no tiene tiempo para terminar el proyecto ahora, asegúrese de saltar al final de este cuaderno y cerrar su punto final. Puede implementarlo nuevamente cuando regrese.

### Paso 7 (nuevamente): use el modelo para la aplicación web
> **TODO**: esta sección completa y la siguiente contienen tareas para completar, principalmente utilizando la consola de AWS.

Hasta ahora hemos estado accediendo a nuestro punto final del modelo construyendo un objeto predictor que usa el punto final y luego simplemente usando el objeto predictor para realizar la inferencia. ¿Qué pasaría si quisiéramos crear una aplicación web que accediera a nuestro modelo? La forma en que se configuran actualmente hace que eso no sea posible, ya que para acceder a un punto final de SageMaker, la aplicación primero tendría que autenticarse con AWS utilizando un rol de IAM que incluye el acceso a los puntos finales de SageMaker. Sin embargo, hay una manera más fácil! Solo necesitamos usar algunos servicios adicionales de AWS.

El diagrama anterior brinda una descripción general de cómo los diversos servicios funcionarán juntos. En el extremo derecho está el modelo que hemos entrenado anteriormente y que se implementa utilizando SageMaker. En el extremo izquierdo está nuestra aplicación web que recopila la crítica de la película de un usuario, la envía y espera un sentimiento positivo o negativo a cambio.

En el medio es donde sucede algo de la magia. Construiremos una función Lambda, que se puede considerar como una función simple de Python que se puede ejecutar siempre que ocurra un evento específico. Le daremos permiso a esta función para enviar y recibir datos desde un punto final de SageMaker.

Por último, el método que usaremos para ejecutar la función Lambda es un nuevo punto final que crearemos usando API Gateway. Este punto final será una url que escuchará los datos que se le enviarán. Una vez que obtiene algunos datos, los pasará a la función Lambda y luego devolverá lo que devuelva la función Lambda. Básicamente, actuará como una interfaz que permite que nuestra aplicación web se comunique con la función Lambda.

### Configuración de una función Lambda
Lo primero que vamos a hacer es configurar una función Lambda. Esta función Lambda se ejecutará siempre que nuestra API pública tenga datos enviados. Cuando se ejecute, recibirá los datos, realizará cualquier tipo de procesamiento que sea necesario, enviará los datos (la revisión) al punto final de SageMaker que hemos creado y luego devolverá el resultado.

#### Parte A: Crear un rol de IAM para la función Lambda
Como queremos que la función Lambda llame a un punto final de SageMaker, debemos asegurarnos de que tenga permiso para hacerlo. Para hacer esto, construiremos un rol que luego podemos asignarle a la función Lambda.

Con la consola de AWS, navegue a la página **IAM** y haga clic en **Roles**. Luego, haga clic en **Crear rol**. Asegúrese de que el **servicio de AWS** es el tipo de entidad de confianza seleccionada y elija **Lambda** como el servicio que utilizará esta función, luego haga clic en **Siguiente: Permisos**.

En el cuadro de búsqueda, escriba sagemaker y seleccione la casilla de verificación junto a la política **AmazonSageMakerFullAccess**. Luego, haga clic en **Siguiente: Revisar**.

Por último, dale un nombre a este rol. Asegúrese de utilizar un nombre que recordará más adelante, por ejemplo, LambdaSageMakerRole. Luego, haga clic en **Crear rol**.

#### Parte B: Crear una función Lambda
Ahora es el momento de crear realmente la función Lambda.

Con la consola de AWS, navegue a la página de AWS Lambda y haga clic en **Crear una función**. Cuando llegue a la página siguiente, asegúrese de que **Autor desde cero** esté seleccionado. Ahora, nombre su función Lambda, usando un nombre que recordará más adelante, por ejemplo sentiment_analysis_func. Asegúrese de que esté seleccionado el tiempo de ejecución **Python 3.6** y luego elija el rol que creó en la parte anterior. Luego, haga clic en **Crear función**.

En la página siguiente verá información sobre la función Lambda que acaba de crear. Si se desplaza hacia abajo, debería ver un editor en el que puede escribir el código que se ejecutará cuando se active su función Lambda. En nuestro ejemplo, utilizaremos el siguiente código.

```python
# We need to use the low-level library to interact with SageMaker since the SageMaker API
# is not available natively through Lambda.
import boto3

def lambda_handler(event, context):

    # The SageMaker runtime is what allows us to invoke the endpoint that we've created.
    runtime = boto3.Session().client('sagemaker-runtime')

    # Now we use the SageMaker runtime to invoke our endpoint, sending the review we were given
    response = runtime.invoke_endpoint(EndpointName = '**ENDPOINT NAME HERE**',    # The name of the endpoint we created
                                       ContentType = 'text/plain',                 # The data format that is expected
                                       Body = event['body'])                       # The actual review

    # The response is an HTTP response whose body contains the result of our inference
    result = response['Body'].read().decode('utf-8')

    return {
        'statusCode' : 200,
        'headers' : { 'Content-Type' : 'text/plain', 'Access-Control-Allow-Origin' : '*' },
        'body' : result
    }
```


Una vez que haya copiado y pegado el código anterior en el editor de código Lambda, reemplace la porción **ENDPOINT NAME HERE** con el nombre del punto final que implementamos anteriormente. Puede determinar el nombre del punto final utilizando la celda de código a continuación.

In [ ]:
predictor.endpoint

Una vez que haya agregado el nombre del punto final a la función Lambda, haga clic en **Guardar**. Su función Lambda ahora está en funcionamiento. A continuación, debemos crear una forma para que nuestra aplicación web ejecute la función Lambda.

### Configuración de API Gateway
Ahora que nuestra función Lambda está configurada, es hora de crear una nueva API utilizando API Gateway que activará la función Lambda que acabamos de crear.

Con la consola de AWS, navegue hasta **Amazon API Gateway** y luego haga clic en **Comenzar**.

En la página siguiente, asegúrese de que **Nueva API** esté seleccionada y asigne un nombre a la nueva API, por ejemplo, sentiment_analysis_api. Luego, haga clic en **Crear API**.

Ahora hemos creado una API, sin embargo, actualmente no hace nada. Lo que queremos que haga es activar la función Lambda que creamos anteriormente.

Seleccione el menú desplegable **Acciones** y haga clic en **Crear método**. Se creará un nuevo método en blanco, seleccione su menú desplegable y seleccione **POST**, luego haga clic en la marca de verificación junto a él.

Para el punto de integración, asegúrese de que **Función Lambda** esté seleccionada y haga clic en **Usar integración de proxy Lambda**. Esta opción asegura que los datos que se envían a la API se envían directamente a la función Lambda sin procesamiento. También significa que el valor de retorno debe ser un objeto de respuesta adecuado, ya que API Gateway tampoco lo procesará.

Escriba el nombre de la **función Lambda** que creó anteriormente en el cuadro de entrada de texto de la función Lambda y luego haga clic en **Guardar**. Haga clic en **Aceptar** en el cuadro emergente que luego aparece, dando permiso a API Gateway para invocar la función Lambda que creó.

El último paso para crear API Gateway es seleccionar el menú desplegable **Acciones** y hacer clic en **Implementar API**. Deberá crear una nueva etapa de implementación y asignarle el nombre que desee, por ejemplo, prod.

Ahora ha configurado con éxito una API pública para acceder a su modelo SageMaker. Asegúrese de copiar o anotar la URL proporcionada para invocar su API pública recién creada, ya que será necesaria en el siguiente paso. Esta URL se puede encontrar en la parte superior de la página, resaltada en azul junto al texto **Invocar URL**.

### Paso 4: Implementar nuestra aplicación web
Ahora que tenemos una API disponible públicamente, podemos comenzar a usarla en una aplicación web. Para nuestros propósitos, hemos proporcionado un archivo html estático simple que puede hacer uso de la API pública que creó anteriormente.

En la carpeta del sitio web debe haber un archivo llamado index.html. Descargue el archivo en su computadora y ábralo en el editor de texto que elija. Debe haber una línea que contenga ****REPLACE WITH PUBLIC API URL****. Reemplace esta cadena con la url que anotó en el último paso y luego guarde el archivo.

Ahora, si abre index.html en su computadora local, su navegador se comportará como un servidor web local y podrá usar el sitio proporcionado para interactuar con su modelo SageMaker.

Si desea ir más allá, puede alojar este archivo html en cualquier lugar que desee, por ejemplo, usando github o alojando un sitio estático en el S3 de Amazon. Una vez que hayas hecho esto, puedes compartir el enlace con cualquier persona que quieras y que también jueguen con él.

> **Nota importante** Para que la aplicación web se comunique con el punto final de SageMaker, el punto final debe implementarse y ejecutarse. Esto significa que está pagando por ello. Asegúrese de que el punto final se esté ejecutando cuando desee usar la aplicación web, pero que lo cierre cuando no lo necesite, de lo contrario terminará con una factura de AWS sorprendentemente grande.

**TODO**: asegúrese de incluir el archivo editado index.html en el envío de su proyecto.

Ahora que su aplicación web funciona, intente jugar con ella y ver qué tan bien funciona.

**Pregunta**: Dé un ejemplo de una revisión que ingresó en su aplicación web. ¿Cuál fue el sentimiento previsto de su revisión de ejemplo?

**Respuesta**:

### Eliminar el punto final
Recuerde siempre cerrar su punto final si ya no lo está usando. Se le cobra por el período de tiempo que el punto final se está ejecutando, por lo que si lo olvida y lo deja encendido, podría terminar con una factura inesperadamente grande.

In [ ]:
predictor.delete_endpoint()